In [1]:
## Bouncing Packet Locator Script

# Date Created: 05/15/23 (Original Script, in MATLAB)
# Date Created: 01/03/24 (This Script)
# Last Modified: 4/30/24

# Author: Max Feinland for Blum Research Group, LASP

# Inputs: start date, end date

# Description: complete driver script. The user will be prompted to specify the dates 
# they would like to see. This script will then print out and plot all of the 
# microbursts found in the specified time period.

# housekeeping
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import dateutil.parser
import sampex
import matplotlib.pylab as plt
import matplotlib.dates as dates
from scipy.signal import find_peaks, peak_widths
import time

C:\Users\maxim\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
## O'Brien

# Date created: 3/21/23 (in MATLAB)
# or 01/04/24 in Python
# Last modified: 02/04/24 (to fix a STUPID indexing issue, smh)
# Author: Max Feinland for Blum Research Group, LASP

# Inputs: sampex package HILT object

# Outputs: starting & ending time of each microburst, plus time intervals

def obrien(data):
    N20 = data['counts'] # count rate sampled every 20 ms
    time20 = data['time'] # time every 20 ms

    # I'm sure there's a more efficient way to do this, but I don't know it
    df = pd.DataFrame({'time': data['time'], 'counts': data['counts']})

    df.set_index('time', inplace=True) # set time column as the index

    # resample the dataframe to 100 ms intervals and sum the counts in each interval
    N100 = df.resample('100ms').sum()

    A500 = N100.rolling(5, center=True).mean() # 5-observation centered rolling mean (over 500 ms)

    condition = np.divide((N100.counts - A500.counts), np.sqrt(1 + A500.counts)) # O'Brien et al 2003
    
    ns = np.argwhere(condition > 10)
    ns = [item[0] for item in ns]

    epsilon = 10; # if two flagged indices are spaced less than this distance apart, 
    # they are probably part of the same microburst

    # initializing
    starts = []
    ends = []

    dn = np.diff(ns) # difference in time between instances of the condition being true

    # finding extended periods of the condition being true
    for i in np.arange(1,len(dn)-10):
        if dn[i] < epsilon and dn[i+1] < epsilon and dn[i-1] >= epsilon: # start condition
            starts.append(ns[i])
            for j in np.arange(i+1, len(dn)-1):
                if dn[j] < epsilon and dn[j+1] >= epsilon:
                    ends.append(ns[j]) # end condition
                    break
        elif dn[i] <= epsilon and i == 1: # start condition (edge case)
            starts.append(ns[i])
            for j in np.arange(i+1, len(dn-1)):
                if dn[j] <= epsilon:
                    ends.append(ns[j])
                    break
        elif i == len(dn): # end condition (edge case)
            ends.append(ns[i])

    if len(starts) > len(ends):
        ends.append(starts[len(starts)-1] + 10)
        
    starts = [x - 2 for x in starts] # pad with 0.2 seconds 
    ends = [x + 10 for x in ends] # pad with 1 second


    def changeCadence(time20, times100):
        # times100 is the list of timestamps for whatever it is: starts, ends, ns
        # times20 is the entire list of time20 for the day in question
        try:
            # try list comprehension first, it's faster
            idx20 = [time20.get_loc(tstmp) for tstmp in times100] 
        except: 
            idx20 = [] # clear and start over
            idx20 = [np.abs((time20 - target).values).argmin() for target in times100] 
            # if a timestamp is missing from time20,
            # you will have to find the minimum timestamp (that is closest in time)
        return idx20

    # reverting indices to 20 ms cadence

    starts20 = changeCadence(time20, N100.index[starts])
    ends20 = changeCadence(time20, N100.index[ends])
    ns20 = changeCadence(time20, N100.index[ns])

    # yay, your output is ready!
    d = dict(); 
    d['st'] = starts20
    d['et'] = ends20
    d['ns'] = ns20
    
    return d

In [13]:
def bouncingPackets_returnspacing(so, eo, h, desired_t):
    # Date created: 5/22/24 (but co-opted from ChorusWaves, which has had it much longer)
    # Last modified: 5/22/24
    # Purpose: takes intervals specified from the O'Brien algorithm and searches for 
    # consistently-spaced, increasing-width, decreasing-in-value, isolated peaks.
    
    
    # Intializing variables
    pksi = []
    st = []
    et = []
    
    # load in HILT data
    t = h.times
    rate = h.counts

    for i in range(len(so)):

        interval = rate[so[i]:eo[i]] # define just one chunk of the rate data, taken from O'Brien
        maxdis = max(interval) - min(interval) # use to generate prominence requirement

        # finding peaks with a high enough prominence
        [pks, _] = find_peaks(interval, prominence=0.25*maxdis, distance=3)
        npks = list(so[i] + loc for loc in pks)
        pksi.extend(npks) # add to "pksi" list, to access later (plotting)

        loc = pd.to_datetime(t[npks])

        dloc = [loc[i + 1] - loc[i] for i in range(len(loc) - 1)] # peak spacing
        dloc = [x.total_seconds() for x in dloc]
        ddloc = np.diff(dloc) # change in peak spacing; must be sufficiently small, i.e.
        # the peaks must be very regularly spaced.

        if len(ddloc) >= 2:
            threshold = 0.05 # lowest allowable change in peak spacing/period

            # find a run of at least 2 indices that meet this criterion. That means 4 peaks
            indices = np.where(np.convolve(np.abs(ddloc) < threshold, \
                                           np.ones(2), mode='valid') == 2)[0]
            if len(indices) == 1: # if there is exactly one microburst
                tstmp_start = loc[indices[0]] - pd.Timedelta(seconds=0.2)
                tstmp_final = loc[indices[0]+3] + pd.Timedelta(seconds=0.2)
                et.extend(np.where(t == tstmp_final)[0])
                st.extend(np.where(t == tstmp_start)[0])
            elif len(indices) > 1:
                for j in range(len(indices)-1):
                    # if you are looking at the first index and there is not a jump, 
                    # that is a start time
                    if j == 0 and indices[j + 1] - indices[j] == 1:
                        tstmp_start = loc[indices[j]] - pd.Timedelta(seconds=0.2)
                        st.extend(np.where(t == tstmp_start)[0])
                    elif j == 0: # otherwise, nothing happens!
                        pass
                    # if previously, you were not in a consecutive streak, 
                    # but now you are, that is a start time
                    elif indices[j+1] - indices[j] == 1 and indices[j] - indices[j-1] > 1:
                        tstmp_start = loc[indices[j]] - pd.Timedelta(seconds=0.2)
                        st.extend(np.where(t == tstmp_start)[0])
                    # if previously, you were in a consecutive streak, but now you are not,
                    # that is an endtime
                    elif indices[j+1] - indices[j] > 1 and indices[j] - indices[j-1] == 1:
                        tstmp_final = loc[indices[j]+3] + pd.Timedelta(seconds=0.2)
                        et.extend(np.where(t == tstmp_final)[0])

                # end condition if the previous one wasn't met
                if len(st) > len(et):
                    tstmp_final = loc[indices[j]+3] + pd.Timedelta(seconds=0.2)
                    et.extend(np.where(t == tstmp_final)[0])


    spacings = []
    for k in range(len(st)):
        if abs(t[st[k]] - desired_t) < timedelta(seconds=4):
            # Find all of the indices in the rate data corresponding to this interval.
            ok_range_for_this_index = np.arange(st[k], et[k]).tolist()
            # Find all the values in pksi that are contained within these indices.
            these_pk_indices = [index for index, value in enumerate(pksi) if 
                                value in ok_range_for_this_index]
            relative_pks = [pksi[x]-st[k] for x in these_pk_indices]
            spacings = np.diff(relative_pks)*.02
            spacings = spacings.tolist()
    return spacings

In [17]:
events = pd.read_csv('events_runonapr29.csv')

date_list = pd.to_datetime(events.t[events.eye==1])
date_list = date_list.reset_index(drop=True)

spacings = []
filename = "spacings.csv"
# for day in date_list:
tn = len(date_list)
for i in range(0, tn):
    day = date_list[i]
    n = i+1
    formatted_date = day.strftime("%B %d, %Y")
    print(f"\nRunning {formatted_date} ({n}/{tn})...")

    h = sampex.HILT(day) # count rate data (thanks Mike!)
    h.load()

    d = obrien(h) # call O'Brien function

    so = d['st']
    eo = d['et']
    ns = d['ns']

    spacingst = bouncingPackets_returnspacing(so, eo, h, day)

#     y = type(np.array([1]))
#     yn = isinstance(spacingst, y)
#     if not yn:
#         print("There is a problem")
#     else:
    spacings.append(spacingst)
    print(spacingst)
    dates = date_list[0:n]
    df = pd.DataFrame({'dt': spacings}, index=dates)
    print(df)
    df.to_csv(filename, encoding='utf-8')

# import os.path
# if os.path.isfile('./' + filename):
#     spacings.to_csv(filename, mode='a', sep=',', index=dates, header=False, encoding='utf-8')
# else:
#     spacings.to_csv(filename, sep=',', index=dates, encoding='utf-8')



Running January 23, 2000 (1/56)...
[0.34, 0.34, 0.34]
                                         dt
t                                          
2000-01-23 03:47:56.320  [0.34, 0.34, 0.34]

Running January 29, 2000 (2/56)...


C:\Users\maxim\Anaconda3\lib\site-packages\sampex\load.py:179: UserWarning: The SAMPEX HILT data is not in order for 2000029.
  warnings.warn(f"The SAMPEX HILT data is not in order for {self.load_date_str}.")


[0.3, 0.34, 0.34]
                                         dt
t                                          
2000-01-23 03:47:56.320  [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460   [0.3, 0.34, 0.34]

Running February 26, 2000 (3/56)...
[0.84, 0.8, 0.8]
                                         dt
t                                          
2000-01-23 03:47:56.320  [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460   [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800    [0.84, 0.8, 0.8]

Running March 01, 2000 (4/56)...
[0.24, 0.22, 0.22]
                                         dt
t                                          
2000-01-23 03:47:56.320  [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460   [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800    [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920  [0.24, 0.22, 0.22]

Running May 25, 2000 (5/56)...
[0.2, 0.6, 0.56, 0.58]
                                             dt
t                                              
2000-01-23 03:47:56.320      [0.34, 0.34, 0.34]
2000-01-29 07:20

C:\Users\maxim\Anaconda3\lib\site-packages\sampex\load.py:179: UserWarning: The SAMPEX HILT data is not in order for 2000271.
  warnings.warn(f"The SAMPEX HILT data is not in order for {self.load_date_str}.")


[0.44, 0.4, 0.36, 0.14]
                                              dt
t                                               
2000-01-23 03:47:56.320       [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460        [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800         [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920       [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040   [0.2, 0.6, 0.56, 0.58]
2000-06-10 10:54:58.460       [0.38, 0.36, 0.34]
2000-06-10 22:47:03.220       [0.68, 0.66, 0.68]
2000-06-11 00:23:20.820       [0.62, 0.62, 0.66]
2000-06-29 01:49:28.960        [0.42, 0.4, 0.44]
2000-09-27 01:07:18.040  [0.44, 0.4, 0.36, 0.14]

Running October 05, 2000 (11/56)...
[0.4, 0.42, 0.42]
                                              dt
t                                               
2000-01-23 03:47:56.320       [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460        [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800         [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920       [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040   [0.2, 0.6, 0.

...

[0.16, 0.2, 0.2]
                                                       dt
t                                                        
2000-01-23 03:47:56.320                [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460                 [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800                  [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920                [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040            [0.2, 0.6, 0.56, 0.58]
2000-06-10 10:54:58.460                [0.38, 0.36, 0.34]
2000-06-10 22:47:03.220                [0.68, 0.66, 0.68]
2000-06-11 00:23:20.820                [0.62, 0.62, 0.66]
2000-06-29 01:49:28.960                 [0.42, 0.4, 0.44]
2000-09-27 01:07:18.040           [0.44, 0.4, 0.36, 0.14]
2000-10-05 11:52:43.200                 [0.4, 0.42, 0.42]
2000-10-29 05:59:28.440                 [0.32, 0.34, 0.3]
2000-11-12 10:45:17.900                [0.54, 0.54, 0.56]
2001-03-20 02:41:25.160                  [0.6, 0.64, 0.6]
2001-03-29 02:49:51.740                [0.56, 0.54, 0.5

[0.22, 0.26, 0.3]
                                                       dt
t                                                        
2000-01-23 03:47:56.320                [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460                 [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800                  [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920                [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040            [0.2, 0.6, 0.56, 0.58]
2000-06-10 10:54:58.460                [0.38, 0.36, 0.34]
2000-06-10 22:47:03.220                [0.68, 0.66, 0.68]
2000-06-11 00:23:20.820                [0.62, 0.62, 0.66]
2000-06-29 01:49:28.960                 [0.42, 0.4, 0.44]
2000-09-27 01:07:18.040           [0.44, 0.4, 0.36, 0.14]
2000-10-05 11:52:43.200                 [0.4, 0.42, 0.42]
2000-10-29 05:59:28.440                 [0.32, 0.34, 0.3]
2000-11-12 10:45:17.900                [0.54, 0.54, 0.56]
2001-03-20 02:41:25.160                  [0.6, 0.64, 0.6]
2001-03-29 02:49:51.740                [0.56, 0.54, 0.

[0.38, 0.38, 0.38]
                                                       dt
t                                                        
2000-01-23 03:47:56.320                [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460                 [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800                  [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920                [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040            [0.2, 0.6, 0.56, 0.58]
2000-06-10 10:54:58.460                [0.38, 0.36, 0.34]
2000-06-10 22:47:03.220                [0.68, 0.66, 0.68]
2000-06-11 00:23:20.820                [0.62, 0.62, 0.66]
2000-06-29 01:49:28.960                 [0.42, 0.4, 0.44]
2000-09-27 01:07:18.040           [0.44, 0.4, 0.36, 0.14]
2000-10-05 11:52:43.200                 [0.4, 0.42, 0.42]
2000-10-29 05:59:28.440                 [0.32, 0.34, 0.3]
2000-11-12 10:45:17.900                [0.54, 0.54, 0.56]
2001-03-20 02:41:25.160                  [0.6, 0.64, 0.6]
2001-03-29 02:49:51.740                [0.56, 0.54, 0

[0.36, 0.32, 0.34]
                                                       dt
t                                                        
2000-01-23 03:47:56.320                [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460                 [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800                  [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920                [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040            [0.2, 0.6, 0.56, 0.58]
2000-06-10 10:54:58.460                [0.38, 0.36, 0.34]
2000-06-10 22:47:03.220                [0.68, 0.66, 0.68]
2000-06-11 00:23:20.820                [0.62, 0.62, 0.66]
2000-06-29 01:49:28.960                 [0.42, 0.4, 0.44]
2000-09-27 01:07:18.040           [0.44, 0.4, 0.36, 0.14]
2000-10-05 11:52:43.200                 [0.4, 0.42, 0.42]
2000-10-29 05:59:28.440                 [0.32, 0.34, 0.3]
2000-11-12 10:45:17.900                [0.54, 0.54, 0.56]
2001-03-20 02:41:25.160                  [0.6, 0.64, 0.6]
2001-03-29 02:49:51.740                [0.56, 0.54, 0

[0.8, 0.8, 0.78]
                                                       dt
t                                                        
2000-01-23 03:47:56.320                [0.34, 0.34, 0.34]
2000-01-29 07:20:13.460                 [0.3, 0.34, 0.34]
2000-02-26 11:26:51.800                  [0.84, 0.8, 0.8]
2000-03-01 17:14:59.920                [0.24, 0.22, 0.22]
2000-05-25 10:16:20.040            [0.2, 0.6, 0.56, 0.58]
2000-06-10 10:54:58.460                [0.38, 0.36, 0.34]
2000-06-10 22:47:03.220                [0.68, 0.66, 0.68]
2000-06-11 00:23:20.820                [0.62, 0.62, 0.66]
2000-06-29 01:49:28.960                 [0.42, 0.4, 0.44]
2000-09-27 01:07:18.040           [0.44, 0.4, 0.36, 0.14]
2000-10-05 11:52:43.200                 [0.4, 0.42, 0.42]
2000-10-29 05:59:28.440                 [0.32, 0.34, 0.3]
2000-11-12 10:45:17.900                [0.54, 0.54, 0.56]
2001-03-20 02:41:25.160                  [0.6, 0.64, 0.6]
2001-03-29 02:49:51.740                [0.56, 0.54, 0.5